In [1]:
!pip install -U ultralytics torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.166
    Uninstalling ultralytics-8.3.166:
      Successfully uninstalled ultralytics-8.3.166


In [2]:
import torch, os, yaml, pathlib, platform, psutil, datetime
from ultralytics import RTDETR, settings, YOLO
print('Torch:', torch.__version__, '| CUDA:', torch.version.cuda, '| Device count:', torch.cuda.device_count())
print('GPU:', torch.cuda.get_device_name(0))
print('RAM:', f'{psutil.virtual_memory().total/1e9:.1f} GB')


Torch: 2.7.1+cu126 | CUDA: 12.6 | Device count: 1
GPU: Tesla T4
RAM: 270.0 GB


In [3]:
# ▣ 2. Custom hyper‑parameters (backbone LR ≍ 0.1× head LR)
hyp = {
    'lr0': 1e-4,      # heads/decoder
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 1e-4,
    'warmup_epochs': 3,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 1e-4,
}
HYPS_FILE = 'custom_hyp.yaml'
with open(HYPS_FILE, 'w') as f:
    yaml.safe_dump(hyp, f)
print('Saved', HYPS_FILE)


Saved custom_hyp.yaml


In [ ]:
# K-Fold Cross Validation Training
torch.backends.cudnn.benchmark = True      # pick fastest convolution kernels

# Configuration
k_fold_dataset_path = "../../datasets/k_fold_cv_balanced"
k_folds = 5
results = []

print(f"Starting {k_folds}-fold cross validation training...")
print(f"Dataset path: {k_fold_dataset_path}")

for fold_idx in range(1, k_folds + 1):
    print(f"\n{'='*60}")
    print(f"TRAINING FOLD {fold_idx}/{k_folds}")
    print(f"{'='*60}")
    
    # Load fresh model for each fold
    model = YOLO("yolo11s.pt")
    
    # Define fold-specific paths
    fold_data_yaml = os.path.join(k_fold_dataset_path, f"fold_{fold_idx}", "data.yaml")
    fold_name = f"yolo11s_fold_{fold_idx}"
    
    print(f"Data YAML: {fold_data_yaml}")
    print(f"Training name: {fold_name}")
    
    # Train on this fold
    try:
        results_fold = model.train(
            data=fold_data_yaml,
            epochs=100,              # will likely stop around epoch 25-40
            imgsz=320,
            batch=32,                # or keep 32 and add grad_accum=2
            amp=True,                # mixed precision
            patience=20,             # early stopping
            optimizer="AdamW",
            save_period=10,
            workers=8,
            cache=True,
            half=True,               # half-precision training
            pretrained=True,         # use pretrained weights
            lr0=1e-4,               # initial learning rate
            lrf=0.01,               # final learning rate (lr0 * lrf)
            momentum=0.937,
            weight_decay=1e-4,
            name=fold_name,
            project="runs/cross_validation_balanced"
        )
        
        # Store results for this fold
        fold_result = {
            'fold': fold_idx,
            'best_epoch': results_fold.best_epoch,
            'best_fitness': results_fold.best_fitness,
            'model_path': model.ckpt_path,
            'final_metrics': {
                'train_loss': getattr(results_fold, 'train_loss', None),
                'val_loss': getattr(results_fold, 'val_loss', None),
                'precision': getattr(results_fold, 'precision', None),
                'recall': getattr(results_fold, 'recall', None),
                'mAP50': getattr(results_fold, 'mAP50', None),
                'mAP50_95': getattr(results_fold, 'mAP50_95', None)
            },
            'status': 'completed'
        }
        results.append(fold_result)
        
        print(f"\nFold {fold_idx} completed successfully!")
        print(f"Best epoch: {results_fold.best_epoch}")
        print(f"Best fitness: {results_fold.best_fitness:.4f}")
        print(f"Model saved: {model.ckpt_path}")
        
    except Exception as e:
        print(f"Error training fold {fold_idx}: {str(e)}")
        fold_result = {
            'fold': fold_idx,
            'status': 'failed',
            'error': str(e)
        }
        results.append(fold_result)

print(f"\n{'='*60}")
print("CROSS VALIDATION SUMMARY")
print(f"{'='*60}")

successful_folds = [r for r in results if r['status'] == 'completed']
failed_folds = [r for r in results if r['status'] == 'failed']

print(f"Successful folds: {len(successful_folds)}/{k_folds}")
print(f"Failed folds: {len(failed_folds)}/{k_folds}")

if successful_folds:
    print("\nSuccessful fold results:")
    for result in successful_folds:
        metrics = result.get('final_metrics', {})
        print(f"  Fold {result['fold']}: Fitness={result['best_fitness']:.4f}, Epoch={result['best_epoch']}")
        if metrics.get('mAP50') is not None:
            print(f"    mAP50={metrics['mAP50']:.4f}, mAP50-95={metrics.get('mAP50_95', 'N/A')}")
    
    # Calculate average metrics
    avg_fitness = sum(r['best_fitness'] for r in successful_folds) / len(successful_folds)
    avg_epoch = sum(r['best_epoch'] for r in successful_folds) / len(successful_folds)
    
    # Calculate average mAP if available
    map50_values = [r['final_metrics'].get('mAP50') for r in successful_folds if r['final_metrics'].get('mAP50') is not None]
    map50_95_values = [r['final_metrics'].get('mAP50_95') for r in successful_folds if r['final_metrics'].get('mAP50_95') is not None]
    
    print(f"\nAverage Results:")
    print(f"  Average fitness: {avg_fitness:.4f}")
    print(f"  Average best epoch: {avg_epoch:.1f}")
    if map50_values:
        print(f"  Average mAP50: {sum(map50_values)/len(map50_values):.4f}")
    if map50_95_values:
        print(f"  Average mAP50-95: {sum(map50_95_values)/len(map50_95_values):.4f}")
        
    # Find best fold by mAP50 if available, otherwise by fitness
    if map50_values:
        best_fold = max(successful_folds, key=lambda x: x['final_metrics'].get('mAP50', 0))
        print(f"\nBest fold by mAP50: Fold {best_fold['fold']} (mAP50={best_fold['final_metrics']['mAP50']:.4f})")
    else:
        best_fold = max(successful_folds, key=lambda x: x['best_fitness'])
        print(f"\nBest fold by fitness: Fold {best_fold['fold']} (fitness={best_fold['best_fitness']:.4f})")

if failed_folds:
    print(f"\nFailed folds:")
    for result in failed_folds:
        print(f"  Fold {result['fold']}: {result['error']}")

print(f"\nAll models saved in: runs/cross_validation/")
print(f"Cross validation complete!")


Starting 5-fold cross validation training...
Dataset path: ../../datasets/k_fold_cv_balanced_subsampled

TRAINING FOLD 1/5
Data YAML: ../../datasets/k_fold_cv_balanced_subsampled/fold_1/data.yaml
Training name: yolo11n_fold_subsampled_1
Ultralytics 8.3.167 🚀 Python-3.11.0rc1 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 14914MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../datasets/k_fold_cv_balanced_subsampled/fold_1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, 

train: Scanning /home/andrea/work/AI-waste-detection/datasets/k_fold_cv_balanced_subsampled/fold_1/train/labels.cache... 5388 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5388/5388 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (1.5GB RAM): 100%|██████████| 5388/5388 [00:01<00:00, 3130.94it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 879.9±318.4 MB/s, size: 27.6 KB)


...

    100/100      1.23G     0.6706     0.4767     0.9961         27        320: 100%|██████████| 169/169 [00:18<00:00,  9.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:04<00:00,  4.90it/s]

                   all       1373       1969      0.844      0.745      0.798      0.595



100 epochs completed in 0.630 hours.
Optimizer stripped from runs/cross_validation_balanced/yolo11n_fold_subsampled_5/weights/last.pt, 5.4MB
Optimizer stripped from runs/cross_validation_balanced/yolo11n_fold_subsampled_5/weights/best.pt, 5.4MB

Validating runs/cross_validation_balanced/yolo11n_fold_subsampled_5/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.0rc1 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 14914MiB)
YOLO11n summary (fused): 100 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:05<00:00,  3.91it/s]


                   all       1373       1969       0.84      0.753        0.8      0.599
                 glass        116        176      0.755      0.733      0.729      0.517
                 metal        444        565      0.859      0.809      0.841      0.571
               organic        156        177      0.877      0.765      0.825      0.661
                 paper        294        320      0.879      0.894       0.93      0.802
               plastic        371        731      0.829      0.564      0.673      0.444
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/cross_validation_balanced/yolo11n_fold_subsampled_5
Error training fold 5: 'DetMetrics' object has no attribute 'best_epoch'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP).

    Attributes:
        names (Dict[int, str]): A dictionary of class names.
        box (Metri

In [ ]:
# Verify K-Fold Dataset Structure
print("Verifying k-fold dataset structure...")
k_fold_dataset_path = "../../datasets/k_fold_cv"

for fold_idx in range(1, 6):  # Check all 5 folds
    fold_dir = os.path.join(k_fold_dataset_path, f"fold_{fold_idx}")
    data_yaml_path = os.path.join(fold_dir, "data.yaml")
    
    if os.path.exists(data_yaml_path):
        with open(data_yaml_path, 'r') as f:
            data_config = yaml.safe_load(f)
        
        # Count files in each split
        splits_info = {}
        for split in ['train', 'val', 'test']:
            images_dir = os.path.join(fold_dir, split, 'images')
            if os.path.exists(images_dir):
                image_count = len([f for f in os.listdir(images_dir) 
                                 if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
                splits_info[split] = image_count
            else:
                splits_info[split] = 0
        
        total = sum(splits_info.values())
        print(f"Fold {fold_idx}: Train={splits_info['train']} ({splits_info['train']/total*100:.1f}%), "
              f"Val={splits_info['val']} ({splits_info['val']/total*100:.1f}%), "
              f"Test={splits_info['test']} ({splits_info['test']/total*100:.1f}%) | Total={total}")
    else:
        print(f"Fold {fold_idx}: ❌ data.yaml not found")

print(f"\nDataset verification complete. Ready for cross-validation training!")

In [ ]:
# ▣ 4. Cross Validation Results Analysis
print("="*60)
print("FINAL CROSS VALIDATION ANALYSIS")
print("="*60)

# Display all fold results
for result in results:
    if result['status'] == 'completed':
        print(f"Fold {result['fold']}: ✓ Fitness={result['best_fitness']:.4f}, Best Model: {result['model_path']}")
    else:
        print(f"Fold {result['fold']}: ✗ Failed - {result.get('error', 'Unknown error')}")

# Optional: Load and test the best model across all folds
if successful_folds:
    best_fold = max(successful_folds, key=lambda x: x['best_fitness'])
    print(f"\nBest performing fold: {best_fold['fold']} (fitness: {best_fold['best_fitness']:.4f})")
    print(f"Best model path: {best_fold['model_path']}")
    
    # Load the best model for inference
    best_model = YOLO(best_fold['model_path'])
    print("Best model loaded and ready for inference!")
else:
    print("No successful folds found!")


In [ ]:
# ▣ 5. Test Best Model on All Test Sets
if successful_folds:
    print("="*60)
    print("TESTING BEST MODEL ON ALL FOLDS")
    print("="*60)
    
    # Use the best model (by mAP50 if available, otherwise by fitness)
    map50_values = [r['final_metrics'].get('mAP50') for r in successful_folds if r['final_metrics'].get('mAP50') is not None]
    if map50_values:
        best_fold = max(successful_folds, key=lambda x: x['final_metrics'].get('mAP50', 0))
        print(f"Using best model from Fold {best_fold['fold']} (mAP50={best_fold['final_metrics']['mAP50']:.4f})")
    else:
        best_fold = max(successful_folds, key=lambda x: x['best_fitness'])
        print(f"Using best model from Fold {best_fold['fold']} (fitness={best_fold['best_fitness']:.4f})")
    
    best_model = YOLO(best_fold['model_path'])
    test_results = []
    
    # Test on each fold's test set
    for fold_idx in range(1, k_folds + 1):
        fold_data_yaml = os.path.join(k_fold_dataset_path, f"fold_{fold_idx}", "data.yaml")
        
        try:
            print(f"\nTesting on Fold {fold_idx} test set...")
            # Validate on this fold's test set
            test_metrics = best_model.val(
                data=fold_data_yaml,
                split='test',  # Use test split
                save_json=True,
                name=f"test_fold_{fold_idx}"
            )
            
            fold_test_result = {
                'fold': fold_idx,
                'mAP50': test_metrics.box.map50 if hasattr(test_metrics.box, 'map50') else None,
                'mAP50_95': test_metrics.box.map if hasattr(test_metrics.box, 'map') else None,
                'precision': test_metrics.box.mp if hasattr(test_metrics.box, 'mp') else None,
                'recall': test_metrics.box.mr if hasattr(test_metrics.box, 'mr') else None
            }
            test_results.append(fold_test_result)
            
            print(f"  mAP50: {fold_test_result['mAP50']:.4f}" if fold_test_result['mAP50'] else "  mAP50: N/A")
            print(f"  mAP50-95: {fold_test_result['mAP50_95']:.4f}" if fold_test_result['mAP50_95'] else "  mAP50-95: N/A")
            
        except Exception as e:
            print(f"  Error testing fold {fold_idx}: {str(e)}")
            test_results.append({'fold': fold_idx, 'error': str(e)})
    
    # Calculate average test performance
    successful_tests = [r for r in test_results if 'error' not in r and r.get('mAP50') is not None]
    if successful_tests:
        avg_test_map50 = sum(r['mAP50'] for r in successful_tests) / len(successful_tests)
        avg_test_map50_95 = sum(r['mAP50_95'] for r in successful_tests if r['mAP50_95']) / len(successful_tests)
        
        print(f"\n{'='*60}")
        print("FINAL TEST RESULTS SUMMARY")
        print(f"{'='*60}")
        print(f"Average Test mAP50: {avg_test_map50:.4f}")
        print(f"Average Test mAP50-95: {avg_test_map50_95:.4f}")
        print(f"Tested on {len(successful_tests)}/{k_folds} folds successfully")
    else:
        print("\nNo successful test results!")
else:
    print("No successful models to test!")
